# SOME BACKGROUND

~ STEP 1 :- First download the latest Python Veriosn 
            https://www.python.org/downloads/

Please Install the downloaded file on your computer using custom Installation settings. 

~ STEP 2 :- After the installation is done , Open  , In MAC - Terminal or WIN - Terminal , and type "Python3 --version"
 for MAC or "Python --version" for WIN and press Enter 

 If , the Terminal , returns with the verion of Python like "Python 3.11.1" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 1 has been accomplished

~STEP 3 :- After we are done  with the STEP 2 , I am using the Jupytr Lab for the tutorials. , type in terminal "pip install jupyter lab" , this will install the latest verison of Jupyter Lab on your system , automatically you will be redirected to the browser 

~ STEP 4 :- To check , if the installation of Jupyter lab has been successful , type "jupyter lab --version" on Terminal. If , the Terminal , returns with the verion of Jupyter Lab like "4.1.2" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 3 has been accomplished

~ STEP 5 :- A library is a collection of code that makes everyday tasks more efficient. For my tutorial i would recommend Installing pandas , numpy - For data analysis , seaborn , matplotlib - For Visualisation.  

Also , i would be Installing a library called "peakutils" , this woulbe be useful for recogisation of Peaks from the Spectra. https://peakutils.readthedocs.io/en/latest/


~ STEP 6 :- For installing the library , go to terminal , and type "pip install pandas" and after installing . Similarly , do this for numpy , seaborn , matplotlin and PeakUtils

~ STEP 7 :- Now , to open the Jupyter Lab seession in the browser , go to terminal and type "jupyter lab" , you will be automatically redirected to the browser in a jupyter lab session. This is where you will start programming . 

NOTE :- To exit the session , press "Ctrl + C" on the termianal , this will end the running session on your system

~ STEP 8 :- In jupyter lab , your lines of codes or chunk of lines of codes  are called as "Cell" , If the "Cell are of type Markdown - Used for Comments / Information " and If the "Cells are of type Code - Used for Programming" 

You can get more in fo here -  https://www.datacamp.com/tutorial/installing-jupyter-notebook


# Importing the Useful librararies. 

Line [1] is completely optional , This is used to Hide the the Warnings in the Output cell. It's recommended to skip the line[1] , and proceed with the code from line[2].

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show

from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

# LOADING THE SPECTRA FILE WHICH WE HAVE

In [3]:
file_path = 'New_Spectra/CR300LA/CR300LA-1.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600

print(max(Sample_Spectra.Intensities))

print (Sample_Spectra)
show (Sample_Spectra_Plot)



26741.599615262785
       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


##### Now , as we see in the figure avove , the spectra have lots of lines , which are very specific to the element present in the Sample . 

Each element has an specific , wavelength associated with it.

Now , i have visited the NIST LIBS database , and download the spectra for all the element in the Periodic Table. The File are updated with that of NIST database as of 17 April 2024.

# CHOOSING THE ELEMENT OF INTEREST

In [4]:

element = 'Fe' # ENTER YOUR ELEMENT OF CHOICE
Reference_Spectra = pd.read_csv(r"NIST/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
print(Reference_Spectra)

Reference_Spectra_Plot = figure(title = 'NIST Reference Library ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Plot.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Plot.width = 1200
Reference_Spectra_Plot.height = 600

show(Reference_Spectra_Plot)

       Wavelength       Spectra
0          169.08  2.630000e-10
1          169.12  5.470000e-09
2          169.16  1.020000e-07
3          169.20  1.700000e-06
4          169.24  2.550000e-05
...           ...           ...
16653      963.36  9.000000e+01
16654      963.40  1.090000e+02
16655      963.44  1.310000e+02
16656      963.48  1.570000e+02
16657      963.52  1.870000e+02

[16658 rows x 2 columns]


# APPROACH FOR PEAK DETECTION IN THE NIST ELEMENT SPECTRA

Now, uptil now , we have selected the Material Sample ,which we want to Interact with and the data is stored in variable "Sample_Spectra"
Also , we have choosen the element , which we want to detect in this 'Sample_Spectra' , and the spectral information for that element is stored in variable "Reference_Spectra" 

Now , from this 'Reference_Spectra' , Let's try to get all the Peaks this Spectra has

I have used a library called as Peak_Utils 

https://peakutils.readthedocs.io/en/latest/reference.html

The detail of the function is given below. 

---------------------------------------------------------------------------------------------------------------------------

NOTE :- Now if we keep the minimum_distance_1 = 0 and threshold_1 = 0 , we get all the peaks in the spectra , Tuning of these parameter are most important as per your signal.

Setting the value in parameter 'minimum_distance_1' , will allows to skip the peaks , which are very close to each other. 
Setting the value in parameter 'threshold_1', will allows to skip the peaks with low intensity and also seperate the Peaks , with could be the noise signal. 

So , its important to choose the value's as per your convenience 

----------------------------------------------------------------------------------------------------------------------------

In [5]:
minimum_distance_1 = 0 
threshold_1 = 0 #looking for a value above 0.01% , so that Peak with very low Intensities can be ignored 

Reference_Peak = indexes(Reference_Spectra.Spectra , min_dist=minimum_distance_1 , thres=threshold_1) 

Selected_Reference_Spectra = Reference_Spectra.iloc[Reference_Peak] #from the indices , lets try to find the Wavelength and Spectra. 

Reference_Spectra_Peaks = figure(title = 'Peaks in NIST Spectra ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Peaks.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Peaks.circle(Selected_Reference_Spectra.Wavelength , Selected_Reference_Spectra.Spectra , color = 'red' , size = 8)

Reference_Spectra_Peaks.width = 1200
Reference_Spectra_Peaks.height = 600

print(Selected_Reference_Spectra)
show(Reference_Spectra_Peaks)



       Wavelength       Spectra
27         170.19  1.630000e+09
44         170.89  3.690000e+08
54         171.30  9.280000e+08
72         172.04  6.800000e+08
85         172.57  3.590000e+08
...           ...           ...
15814      917.87  6.420000e+05
15875      920.38  7.010000e+05
16155      942.89  1.540000e+05
16347      950.78  2.200000e+04
16549      959.08  1.580000e+04

[333 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [6]:
Reference_Spectra_List = []

for iteam in Selected_Reference_Spectra.Wavelength:
    iteam = float(iteam)
    if iteam > 187.0:        #Only those iteam will be selected , whose wavelength would be greater than 188 nm ,  as our Handheld LIBS range starts from 188nm
        Reference_Spectra_List.append(iteam)
    
print(Reference_Spectra_List) #These are the wavelengths of Peaks , which we are interested in the NIST Spectra
Length_Reference_Spectra_List = (len(Reference_Spectra_List))   #Used for Calculating Percentage Detection

[187.73, 188.88, 189.54, 189.79, 190.49, 191.06, 192.58, 193.48, 194.06, 194.84, 195.95, 196.61, 197.84, 198.62, 200.02, 201.05, 201.91, 203.26, 203.63, 204.09, 205.11, 205.52, 206.43, 206.72, 208.03, 208.73, 209.34, 210.99, 211.69, 213.08, 213.66, 215.02, 216.21, 216.7, 217.56, 218.38, 219.21, 219.62, 220.03, 220.77, 221.42, 222.04, 222.82, 223.4, 223.81, 224.88, 225.12, 225.57, 226.4, 228.0, 229.81, 230.59, 231.2, 233.26, 233.79, 234.37, 234.78, 236.01, 236.46, 237.41, 238.19, 238.85, 239.59, 240.53, 241.07, 242.42, 243.28, 243.94, 244.52, 246.2, 247.19, 248.3, 249.24, 249.9, 251.17, 252.65, 253.8, 254.91, 256.27, 257.71, 258.57, 259.23, 259.92, 260.75, 261.2, 263.09, 265.02, 265.8, 266.54, 268.47, 269.25, 270.44, 271.43, 272.74, 273.98, 274.8, 275.54, 276.77, 278.37, 279.4, 279.89, 280.55, 281.33, 283.22, 284.0, 284.7, 285.77, 287.37, 288.36, 289.55, 290.78, 291.85, 292.67, 293.7, 294.73, 295.38, 296.74, 297.23, 298.47, 299.45, 300.19, 302.08, 303.72, 304.75, 305.82, 306.68, 307.67,

# 0.0333 is the sprectral resolution of the Handheld LIBS Device

Now , for the two wavelengths from two different data source , to exactly match with each other , is going to be difficult because of the spectral resolution of wavelength - for these two data source are very different from each other.

Now we know that the Spectral Resolution of Handheld LIBS Device is 0.0333. 

Now , we have a list of all wavelengths , detected for Peaks in Reference Spectra , Now for each element lets try to create a min and max of each element in the list. 'Reference_Spectra_Min_Max' is list consisting of a sublist , the sublist have a min and max value or a range of element in the list.


Eg : - A peak of 254.8 in NIST Spectra , can be found at and between [254.7668 ~ 254.8332] in the Handheld LIBS Spectra , because the resolution of wavelength in NIST Spectra and Handheld LIBS Spectra is different. 


In [7]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.0332, num + 0.0332] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

[[187.6968, 187.76319999999998], [188.8468, 188.9132], [189.5068, 189.57319999999999], [189.7568, 189.82319999999999], [190.45680000000002, 190.5232], [191.0268, 191.0932], [192.54680000000002, 192.6132], [193.4468, 193.51319999999998], [194.0268, 194.0932], [194.8068, 194.8732], [195.9168, 195.98319999999998], [196.57680000000002, 196.6432], [197.8068, 197.8732], [198.5868, 198.6532], [199.98680000000002, 200.0532], [201.01680000000002, 201.0832], [201.8768, 201.9432], [203.2268, 203.29319999999998], [203.5968, 203.6632], [204.0568, 204.1232], [205.07680000000002, 205.1432], [205.48680000000002, 205.5532], [206.3968, 206.4632], [206.6868, 206.7532], [207.9968, 208.0632], [208.6968, 208.76319999999998], [209.3068, 209.3732], [210.95680000000002, 211.0232], [211.6568, 211.7232], [213.04680000000002, 213.1132], [213.6268, 213.6932], [214.98680000000002, 215.0532], [216.17680000000001, 216.2432], [216.6668, 216.73319999999998], [217.5268, 217.5932], [218.3468, 218.4132], [219.176800000000

# Pre-Processing the Spectra for Handheld LIBS to improve the accuracy of Peak Detection 

As we see in our raw spectra , there are lot of small peaks , which really aren't emission peaks , but are the noise in the signal. To tackle that Smoothing the Spectra is usually done. 

I am sing the Savitsky Golay , for smoothing my spectra. 

# Noise Estimation
As we see in the Handheld LIBS Spectra , It's difficult to find or detect , what is the noise and what is the signal.

But we have an information , that the range of Spectrometer in hLIBS starts from 190nm , So we can say that the signal detected uptil 190nm could be classified as noise.

In [8]:
noise_list = [] 
noise_start = []

noise_list = Sample_Spectra.loc[Sample_Spectra['wavelength'] < 190.0 , 'Intensities']

RMS_intensity_noise = max(noise_list) #np.sqrt(np.mean((noise_list)**2)) #This is the max Intensity of the noise signal
print(RMS_intensity_noise)  



55.27522300553263


In [9]:
#METHOD  - Scipy

Threshold = 4 #Noise Intensity + Threshold Value for signal filtering
height_2 = None
Peak_Prominenace = (RMS_intensity_noise * Threshold)  #Uses S/N ratio , as comapred to other peaks
minimum_distance_3 = None   #minimum distance between two peaks

Sample_Peak , _ = fp(Sample_Spectra.Intensities,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3)
Selected_Sample_Spectra = Sample_Spectra.iloc[Sample_Peak] 

Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Peaks.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.Intensities , color = 'red' , size = 8)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

print(Selected_Sample_Spectra)
show(Sample_Spectra_Peaks)

       wavelength   Intensities
392    193.066667   2007.111968
570    199.000000    518.745688
677    202.566667  23665.727339
722    204.066667   1191.364587
786    206.200000  23964.807377
...           ...           ...
20224  854.133333    473.413186
20603  866.766667    920.009176
20640  868.000000    366.367580
21967  912.233333   5193.060147
22273  922.433333   1749.920994

[175 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [10]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


[193.06666666666592, 198.9999999999989, 202.56666666666536, 204.0666666666653, 206.1999999999985, 206.43333333333183, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.6999999999983, 210.03333333333163, 210.2333333333316, 210.4333333333316, 213.86666666666477, 232.699999999997, 233.2333333333303, 233.7999999999969, 234.3666666666636, 234.83333333333024, 235.96666666666349, 236.46666666666343, 237.33333333333007, 237.93333333333004, 238.16666666666336, 238.83333333333, 239.53333333332995, 239.8999999999966, 240.46666666666323, 240.6666666666632, 241.03333333332984, 241.2999999999965, 242.39999999999645, 242.9666666666631, 243.43333333332973, 243.8999999999964, 244.466666666663, 245.8333333333296, 246.1333333333296, 247.8666666666628, 248.2999999999961, 249.0333333333294, 249.2999999999961, 249.93333333332936, 250.199999999996, 251.1333333333293, 251.5666666666626, 252.26666666666256, 252.5333333333292, 252.89999999999583, 253.3666666666625, 253.63333333332915, 253.8333333333

# So far so now , i have two list with me 

1) Sample_Spectra_List = Containg wavelngths of all Peaks from data of Handheld LIBS device
2) Reference_Spectra_Min_Max = Containing a range of wavelength , for all the peak detected in the Reference Sample

   

In [11]:
print(Sample_Spectra_List)

[193.06666666666592, 198.9999999999989, 202.56666666666536, 204.0666666666653, 206.1999999999985, 206.43333333333183, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.6999999999983, 210.03333333333163, 210.2333333333316, 210.4333333333316, 213.86666666666477, 232.699999999997, 233.2333333333303, 233.7999999999969, 234.3666666666636, 234.83333333333024, 235.96666666666349, 236.46666666666343, 237.33333333333007, 237.93333333333004, 238.16666666666336, 238.83333333333, 239.53333333332995, 239.8999999999966, 240.46666666666323, 240.6666666666632, 241.03333333332984, 241.2999999999965, 242.39999999999645, 242.9666666666631, 243.43333333332973, 243.8999999999964, 244.466666666663, 245.8333333333296, 246.1333333333296, 247.8666666666628, 248.2999999999961, 249.0333333333294, 249.2999999999961, 249.93333333332936, 250.199999999996, 251.1333333333293, 251.5666666666626, 252.26666666666256, 252.5333333333292, 252.89999999999583, 253.3666666666625, 253.63333333332915, 253.8333333333

In [12]:
print(Reference_Spectra_Min_Max)

[[187.6968, 187.76319999999998], [188.8468, 188.9132], [189.5068, 189.57319999999999], [189.7568, 189.82319999999999], [190.45680000000002, 190.5232], [191.0268, 191.0932], [192.54680000000002, 192.6132], [193.4468, 193.51319999999998], [194.0268, 194.0932], [194.8068, 194.8732], [195.9168, 195.98319999999998], [196.57680000000002, 196.6432], [197.8068, 197.8732], [198.5868, 198.6532], [199.98680000000002, 200.0532], [201.01680000000002, 201.0832], [201.8768, 201.9432], [203.2268, 203.29319999999998], [203.5968, 203.6632], [204.0568, 204.1232], [205.07680000000002, 205.1432], [205.48680000000002, 205.5532], [206.3968, 206.4632], [206.6868, 206.7532], [207.9968, 208.0632], [208.6968, 208.76319999999998], [209.3068, 209.3732], [210.95680000000002, 211.0232], [211.6568, 211.7232], [213.04680000000002, 213.1132], [213.6268, 213.6932], [214.98680000000002, 215.0532], [216.17680000000001, 216.2432], [216.6668, 216.73319999999998], [217.5268, 217.5932], [218.3468, 218.4132], [219.176800000000

# Finally , My  idea is see , if each iteam in (Reference_Spectra_Min_Max) , matches with each element in (Sample_Spectra_List) , and then caluclate the exact matching Percentage 


In [13]:
plot_spectra_wavelength_list=[]
plot_reference_wavelength_list = []
count = 0
for cell in Reference_Spectra_Min_Max: 
    for iteam in Sample_Spectra_List:
        if iteam >= cell[0] and iteam <= cell[1]:
            count = count + 1 #for counting
            plot_spectra_wavelength_list.append(iteam)
            plot_reference_wavelength_list.append(cell[0] + 0.0333)

plot_spectra_intensities_list=[]
for iteam in plot_spectra_wavelength_list:
    Inte = Selected_Sample_Spectra.loc[Selected_Sample_Spectra['wavelength'] == iteam, 'Intensities'].iloc[0]
    plot_spectra_intensities_list.append(Inte)
#print(plot_spectra_intensities_list)

#print(plot_spectra_wavelength_list)

print('The total peaks detected were ' + str(count) + '\n' )

print('The following peaks , for Reference Sample was detected  :- '  )
print(plot_reference_wavelength_list )

print('\n')
print('The Sample Peaks detected from the sample were:-')
spectra_df = pd.DataFrame({'wavelength': plot_spectra_wavelength_list, 'intensities': plot_spectra_intensities_list})
print(spectra_df)  



Normalization_Intensity = max(spectra_df.intensities)

# ###########################################################################

matching_row = Sample_Spectra[Sample_Spectra['Intensities'] == Normalization_Intensity]
print(matching_row)

The total peaks detected were 25

The following peaks , for Reference Sample was detected  :- 
[204.0901, 206.4301, 208.7301, 233.2601, 233.7901, 234.3701, 236.4601, 238.1901, 238.8501, 242.4201, 248.30010000000001, 254.9101, 259.9201, 263.09009999999995, 271.4301, 272.7401, 275.5401, 358.13009999999997, 372.01009999999997, 382.0001, 385.9401, 404.5601, 422.6801, 430.77009999999996, 438.33009999999996]


The Sample Peaks detected from the sample were:-
    wavelength   intensities
0   204.066667   1191.364587
1   206.433333  17513.347573
2   208.700000  12630.515078
3   233.233333   1047.745769
4   233.800000    759.287103
5   234.366667   1936.378122
6   236.466667    950.641002
7   238.166667   3896.108254
8   238.833333   1324.281072
9   242.400000    617.388039
10  248.300000    969.598893
11  254.933333    865.682459
12  259.900000   4878.244163
13  263.100000   2488.137651
14  271.400000    721.240618
15  272.733333    666.628219
16  275.566667   2805.378975
17  358.133333    512

# RESULTS

In [14]:
percentage = (count / Length_Reference_Spectra_List) * 100
percentage

print (' Ananlysis of ' + element + ' was done , and the ' + str(count) +  ' number of spectral lines from the material sample ' + ' matched with the ' + str(Length_Reference_Spectra_List) + ' choosen Reference element of '+ element + ' therefore , we can say' ,  str(percentage) + '%' + ' matched as per the parameters set by the user')

 Ananlysis of Fe was done , and the 25 number of spectral lines from the material sample  matched with the 312 choosen Reference element of Fe therefore , we can say 8.012820512820513% matched as per the parameters set by the user


Find the Correponding Intensities , for the wavelengths available in "plot_spectra_wavelength_list" and also for the wavelengths available in "plot_refernce_intensities_list"

# Generate a Interactive Plot

In [15]:
p = figure(title = 'Peak Detection' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensities')
p.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
p.circle(spectra_df.wavelength, spectra_df.intensities , color = 'green' , size = 5)

p.width = 1200
p.height = 600

show(p)

